The objective is to calibrate parameters $\rho$ and $\alpha$ of the CES utility function:


$$u(x,T) = [\alpha T_{int}^\rho + (1 - \alpha) x^\rho ]^{1\rho}$$


We know rel

In [2]:
import os
import pandas as pd
from numpy import log
from scipy.optimize import curve_fit
from project.model import get_inputs
from project.thermal import heat_intensity
from project.utils import reindex_mi

In [3]:
# 'buildingstock_sdes2018_simple_1.csv'
resirf_inputs = get_inputs(variables=['buildings', 'income', 'energy_prices'],
                           building_stock=os.path.join('project', 'input', 'stock', 'buildingstock_example.csv'))
buildings = resirf_inputs['buildings']
income = resirf_inputs['income']
income.index.names = ['Income tenant']
price = resirf_inputs['energy_prices'].iloc[0, :]


2022-12-20 19:31:12,645 - 50565 - log_ - INFO - Creating AgentBuildings object


In [4]:
heat_consumption = buildings.heating_consumption() * buildings.surface
energy_bill = buildings.energy_bill(price, heat_consumption)
income = reindex_mi(income, energy_bill.index)
budget_share = energy_bill / income
heating_intensity = heat_intensity(budget_share)

In [5]:
def demand_ces(X, sigma, alpha):
    """Demand function derived from ces utility function.


    Represents the tradeoff between thermal comfort and energy expenditure.

    Parameters
    ----------
    X: tuple
        Income, Price
    sigma: elasticity of subsitution
    alpha

    Returns
    -------

    """
    income, energy_bill = X
    s = ((alpha / energy_bill) ** sigma) * (income / (alpha**sigma * energy_bill ** (1 - sigma) + (1 - alpha) ** sigma))
    x = ((1 - alpha) ** sigma) * (income / (alpha**sigma * energy_bill**(1 - sigma) + (1 - alpha)**sigma))
    return s, x


In [6]:
def demand_ces_homogenous(X, sigma):
    """Demand function derived from ces utility function.


    Represents the tradeoff between thermal comfort and energy expenditure.

    Parameters
    ----------
    X: tuple
        Income, Price
    sigma: elasticity of subsitution
    alpha

    Returns
    -------

    """
    income, energy_bill = X
    return energy_bill**(sigma - 1) * income / (energy_bill**sigma + 1)


def rho_ces(income, energy_bill, s):
    P = income / 1
    X = s / (income - energy_bill * s)
    return log(P) / log(X) + 1



In [27]:
demand_ces_homogenous((10000, 2000), -0.1)

1.593133365791219

In [28]:
demand_ces_homogenous((10000, 3000), -0.1)

1.0329644230181068

In [20]:
energy_bill

Existing  Occupancy status  Income owner  Income tenant  Housing type   Heating system                  Wall  Floor  Roof  Windows
True      Owner-occupied    D1            D1             Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6         201.411230
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5        1835.747981
                                                         Single-family  Natural gas-Standard boiler     2.5   2.5    2.5   4.5        5063.773049
                                                                        Natural gas-Performance boiler  0.1   0.2    0.1   1.6         441.618745
                            D10           D10            Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6         201.411230
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5        1835.

In [21]:
sigma

Existing  Occupancy status  Income owner  Income tenant  Housing type   Heating system                  Wall  Floor  Roof  Windows
True      Owner-occupied    D1            D1             Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6       -0.762444
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5       -0.325735
                                                         Single-family  Natural gas-Standard boiler     2.5   2.5    2.5   4.5       -0.231714
                                                                        Natural gas-Performance boiler  0.1   0.2    0.1   1.6       -0.564496
                            D10           D10            Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6       -1.125251
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5       -0.550711
           

In [19]:
P = energy_bill / 1
X = heating_intensity / (income - energy_bill * heating_intensity)
rho = log(P) / log(X) + 1
sigma = rho / (rho - 1)
print(demand_ces_homogenous((income, energy_bill), sigma))
p0 = sigma.mean()
sigma_fit, _, _, msg, _ = curve_fit(demand_ces_homogenous, (income, energy_bill), heating_intensity, p0=p0, method='lm', full_output=True)
print(msg)
rho_fit =  sigma_fit / (sigma_fit - 1)
print(rho_fit, sigma_fit)
print(sigma_fit[0])

Existing  Occupancy status  Income owner  Income tenant  Housing type   Heating system                  Wall  Floor  Roof  Windows
True      Owner-occupied    D1            D1             Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6        0.856926
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5        0.434843
                                                         Single-family  Natural gas-Standard boiler     2.5   2.5    2.5   4.5        0.241043
                                                                        Natural gas-Performance boiler  0.1   0.2    0.1   1.6        0.706972
                            D10           D10            Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6        1.301570
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5        0.879487
           

In [8]:
sigma

Existing  Occupancy status  Income owner  Income tenant  Housing type   Heating system                  Wall  Floor  Roof  Windows
True      Owner-occupied    D1            D1             Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6       -0.762444
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5       -0.325735
                                                         Single-family  Natural gas-Standard boiler     2.5   2.5    2.5   4.5       -0.231714
                                                                        Natural gas-Performance boiler  0.1   0.2    0.1   1.6       -0.564496
                            D10           D10            Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6       -1.125251
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5       -0.550711
           

In [17]:
s, x = demand_ces((income, energy_bill), -0.5, 0.001)
print(s, x)
print(energy_bill * s + x, income)

Existing  Occupancy status  Income owner  Income tenant  Housing type   Heating system                  Wall  Floor  Roof  Windows
True      Owner-occupied    D1            D1             Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6         49.798062
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5          5.463711
                                                         Single-family  Natural gas-Standard boiler     2.5   2.5    2.5   4.5          1.980736
                                                                        Natural gas-Performance boiler  0.1   0.2    0.1   1.6         22.711821
                            D10           D10            Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6        510.790095
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5         56.042529

In [18]:
s, x = demand_ces((income, energy_bill), 0.5, 0.1)
print(s, x)
print(energy_bill * s + x, income)

Existing  Occupancy status  Income owner  Income tenant  Housing type   Heating system                  Wall  Floor  Roof  Windows
True      Owner-occupied    D1            D1             Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6         41.108738
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5          5.106184
                                                         Single-family  Natural gas-Standard boiler     2.5   2.5    2.5   4.5          1.900610
                                                                        Natural gas-Performance boiler  0.1   0.2    0.1   1.6         19.874652
                            D10           D10            Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6        421.661711
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5         52.375295

In [11]:
heating_intensity_fit = demand_ces_homogenous((income, energy_bill), sigma_fit[0])
heating_intensity_fit

Existing  Occupancy status  Income owner  Income tenant  Housing type   Heating system                  Wall  Floor  Roof  Windows
True      Owner-occupied    D1            D1             Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6        0.154768
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5        0.001539
                                                         Single-family  Natural gas-Standard boiler     2.5   2.5    2.5   4.5        0.000185
                                                                        Natural gas-Performance boiler  0.1   0.2    0.1   1.6        0.030104
                            D10           D10            Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6        1.587486
                                                                        Natural gas-Standard boiler     2.5   2.5    2.5   4.5        0.015787
           

In [12]:
pd.concat((heating_intensity, heating_intensity_fit), axis=1)

0  \
Existing Occupancy status Income owner Income tenant Housing type  Heating system                 Wall Floor Roof Windows             
True     Owner-occupied   D1           D1            Multi-family  Natural gas-Performance boiler 0.1  0.2   0.1  1.6      0.856926   
                                                                   Natural gas-Standard boiler    2.5  2.5   2.5  4.5      0.434843   
                                                     Single-family Natural gas-Standard boiler    2.5  2.5   2.5  4.5      0.241043   
                                                                   Natural gas-Performance boiler 0.1  0.2   0.1  1.6      0.706972   
                          D10          D10           Multi-family  Natural gas-Performance boiler 0.1  0.2   0.1  1.6      1.301570   
                                                                   Natural gas-Standard boiler    2.5  2.5   2.5  4.5      0.879487   
                                                     Single-family Natural gas-Standard boiler    2.5  2.5   2.5  4.5      0.685687   
                                                                   Natural gas-Performance boiler 0.1  0.2   0.1  1.6      1.151616   

                                                                                                                                  1  
Existing Occupancy status Income owner Income tenant Housing type  Heating system                 Wall Floor Roof Windows            
True     Owner-occupied   D1           D1            Multi-family  Natural gas-Performance boiler 0.1  0.2   0.1  1.6      0.154768  
                                                                   Natural gas-Standard boiler    2.5  2.5   2.5  4.5      0.001539  
                                                     Single-family Natural gas-Standard boiler    2.5  2.5   2.5  4.5      0.000185  
                                                                   Natural gas-Performance boiler 0.1  0.2   0.1  1.6      0.030104  
                          D10          D10           Multi-family  Natural gas-Performance boiler 0.1  0.2   0.1  1.6      1.587486  
                                                                   Natural gas-Standard boiler    2.5  2.5   2.5  4.5      0.015787  
                                                     Single-family Natural gas-Standard boiler    2.5  2.5   2.5  4.5      0.001898  
                                                                   Natural gas-Performance boiler 0.1  0.2   0.1  1.6      0.308781

In [13]:
demand_ces((income, energy_bill), rho_fit[0], 0.5)


(Existing  Occupancy status  Income owner  Income tenant  Housing type   Heating system                  Wall  Floor  Roof  Windows
 True      Owner-occupied    D1            D1             Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6         46.162402
                                                                         Natural gas-Standard boiler     2.5   2.5    2.5   4.5          5.318356
                                                          Single-family  Natural gas-Standard boiler     2.5   2.5    2.5   4.5          1.947990
                                                                         Natural gas-Performance boiler  0.1   0.2    0.1   1.6         21.546643
                             D10           D10            Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6        473.498292
                                                                         Natural gas-Standard boiler     2.5   2.5    2.5   4.5         54

In [14]:
demand_ces((income, energy_bill), sigma_fit[0], 0.5)

(Existing  Occupancy status  Income owner  Income tenant  Housing type   Heating system                  Wall  Floor  Roof  Windows
 True      Owner-occupied    D1            D1             Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6         49.797843
                                                                         Natural gas-Standard boiler     2.5   2.5    2.5   4.5          5.463712
                                                          Single-family  Natural gas-Standard boiler     2.5   2.5    2.5   4.5          1.980736
                                                                         Natural gas-Performance boiler  0.1   0.2    0.1   1.6         22.711830
                             D10           D10            Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6        510.787843
                                                                         Natural gas-Standard boiler     2.5   2.5    2.5   4.5         56

In [15]:
p0 = (0.5, 0.5)
demand_ces((income, energy_bill), p0[0], p0[1])

(Existing  Occupancy status  Income owner  Income tenant  Housing type   Heating system                  Wall  Floor  Roof  Windows
 True      Owner-occupied    D1            D1             Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6         46.520651
                                                                         Natural gas-Standard boiler     2.5   2.5    2.5   4.5          5.339100
                                                          Single-family  Natural gas-Standard boiler     2.5   2.5    2.5   4.5          1.953287
                                                                         Natural gas-Performance boiler  0.1   0.2    0.1   1.6         21.680230
                             D10           D10            Multi-family   Natural gas-Performance boiler  0.1   0.2    0.1   1.6        477.172942
                                                                         Natural gas-Standard boiler     2.5   2.5    2.5   4.5         54

In [16]:
curve_fit(demand_ces, (income, energy_bill), heating_intensity, p0=p0, method='lm', full_output=True)


ValueError: object too deep for desired array

error: Result from function call is not a proper array of floats.

In [ ]:
curve_fit(demand_ces, (income, energy_bill), heating_intensity, p0=p0, method='trf')

In [ ]:
curve_fit(demand_ces, (income, energy_bill), heating_intensity, p0=p0, method='dogbox')